# Detecting Speed Limit Signs with Dinov2: a Supervised-Learning Study

### Imports

In [52]:
import torch
from ultralytics import YOLO
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

### Train The Model

In [46]:
# Load pretrained YOLO11n classification model
model = YOLO("yolo11n-cls.pt")

# Train on Images folder
# The Images folder should contain subdirectories for each class:
# Images/
#   speed_limit/
#   no_speed_limit/
results = model.train(data="Images", epochs=20, imgsz=64)

Ultralytics 8.3.234 🚀 Python-3.13.0 torch-2.9.1 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Images, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, pro

### Test The Model and Save Results

In [47]:
results = model("Images_split/test")
csv_filename = "test_predictions.csv"
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename", "predicted_class", "confidence"])
    for r in results:
        fname = r.path.split("/")[-1]  # get only filename
        class_idx = r.probs.top1
        class_name = r.names[class_idx]
        confidence = r.probs.top1conf.item()
        writer.writerow([fname, class_name, confidence])


image 1/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00516.jpg: 64x64 no_speed_limit 1.00, speed_limit 0.00, 6.0ms
image 2/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00517.jpg: 64x64 no_speed_limit 0.99, speed_limit 0.01, 1.8ms
image 3/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00521.jpg: 64x64 no_speed_limit 1.00, speed_limit 0.00, 2.3ms
image 4/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00522.jpg: 64x64 no_speed_limit 1.00, speed_limit 0.00, 2.1ms
image 5/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00523.jpg: 64x64 no_speed_limit 1.00, speed_limit 0.00, 2.0ms
image 6/85 /Users/ryanbirmingham/School Work/Fall2025/INST414/INST_414_Module_6/Images_split/test/00524.jpg: 64x64 no_speed_limit 0.97, speed_limit 0.03, 1.5ms
image 7/85 /Users/ryanbirmingham/School

In [62]:
speed_limit_truth = os.listdir("Images_split/test_truth/speed_limit")
no_speed_limit_truth = os.listdir("Images_split/test_truth/no_speed_limit")

df = pd.read_csv("test_predictions.csv")
df["truth"] = df["filename"].apply(lambda x: "speed_limit" if x in speed_limit_truth else "no_speed_limit")
df["correct"] = df["predicted_class"] == df["truth"]

print(df["correct"].value_counts())
print(df["correct"].value_counts(normalize=True))
print("--------------------------------")
print("These are the correct predictions:")
correct = df[df["correct"] == True]
print(correct[["filename", "predicted_class", "truth"]][:5])
print("--------------------------------")
print("These are the incorrect predictions:")
incorrect = df[df["correct"] == False]
print(incorrect[["filename", "predicted_class", "truth"]][:5])

print("Unique in truth:", df["truth"].unique())
print("Unique in predicted:", df["predicted_class"].unique())


labels = ["no_speed_limit", "speed_limit"]
cm = confusion_matrix(df["truth"], df["predicted_class"], labels=labels)
df["truth"] = df["truth"].str.strip()
df["predicted_class"] = df["predicted_class"].str.strip()
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

correct
True     76
False     9
Name: count, dtype: int64
correct
True     0.894118
False    0.105882
Name: proportion, dtype: float64
--------------------------------
These are the correct predictions:
    filename predicted_class           truth
0  00516.jpg  no_speed_limit  no_speed_limit
1  00517.jpg  no_speed_limit  no_speed_limit
2  00521.jpg  no_speed_limit  no_speed_limit
3  00522.jpg  no_speed_limit  no_speed_limit
4  00523.jpg  no_speed_limit  no_speed_limit
--------------------------------
These are the incorrect predictions:
     filename predicted_class           truth
14  00559.jpg     speed_limit  no_speed_limit
38  00622.jpg  no_speed_limit     speed_limit
40  00642.jpg     speed_limit  no_speed_limit
41  00643.jpg     speed_limit  no_speed_limit
54  00676.jpg  no_speed_limit     speed_limit
Unique in truth: ['no_speed_limit' 'speed_limit']
Unique in predicted: ['no_speed_limit' 'speed_limit']


<Figure size 500x400 with 2 Axes>